In [ ]:
#Import libraries

# data anlysis
import numpy as np
import pandas as pd

# data visualization
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# data preparation for modelling
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

import time
import tensorflow as tf
import pandas as pd
import os
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from keras import Sequential
from keras import layers
from keras import backend as K
from keras.layers.core import Dense
from keras import regularizers
from keras.layers import Dropout , BatchNormalization
from keras.constraints import max_norm

# ensure comparability of different runs
np.random.seed(2020)

In [ ]:
# df = pd.read_pickle("../input/handling-imbalanced-data-eda-small-fe/df_for_use.pkl")
# df_fe = pd.read_pickle("../input/handling-imbalanced-data-eda-small-fe/df_fe.pkl")

In [ ]:
df = pd.read_pickle('../input/searching-for-bad-loan-data-preprocessing/df_pp.pkl')

In [ ]:
# X = df.drop('Loan_status', axis=1)
# # x_dd = X.copy()
# # from sklearn.preprocessing import StandardScaler
# # from sklearn.preprocessing import MinMaxScaler
# # minMaxScaler = MinMaxScaler()
# # minMaxScaler.fit(X)
# # X_scaled = minMaxScaler.transform(X)

# # X= pd.DataFrame(X_scaled, columns=x_dd.columns)
# # X = reduce_mem_usage(X)
# y = df['Loan_status']

# from sklearn import preprocessing as pp
# featuresToScale = X.columns
# sX = pp.MinMaxScaler(copy=True)
# X.loc[:,featuresToScale] = sX.fit_transform(X[featuresToScale])


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 2020, stratify = y)

In [ ]:
data = df.copy()

In [ ]:
data.head()

In [ ]:
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent',)),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

Categorical Features in the dataset:, home_ownership, term, application_type, purpose, region.

Ordinal Features in the dataset :emp_length_int, grade, interest_payments, income_category

Continous Features in the dataset: loan_amount, annual_inc, interest_rate, dti, total_pymnt, recoveries, installment

In [ ]:
data.columns

In [ ]:
cat_cols = [ 'home_ownership_cat', 'term_cat', 'application_type_cat', 'purpose_cat', 'region','emp_length_int', 'grade_cat', 'interest_payment_cat', 'income_cat' ]

In [ ]:
num_cols = ['loan_amount', 'annual_inc', 'interest_rate', 'dti', 'total_pymnt', 'recoveries', 'installment']

In [ ]:
data.head()

In [ ]:
X = data.drop('Loan_status', axis=1)
# x_dd = X.copy()
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# minMaxScaler = MinMaxScaler()
# minMaxScaler.fit(X)
# X_scaled = minMaxScaler.transform(X)

# X= pd.DataFrame(X_scaled, columns=x_dd.columns)
# X = reduce_mem_usage(X)
y = data['Loan_status']

from sklearn import preprocessing as pp
featuresToScale = X.columns
sX = pp.MinMaxScaler(copy=True)
X.loc[:,featuresToScale] = sX.fit_transform(X[featuresToScale])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 2020, stratify = y)

In [ ]:
input_dim = X_train.shape[1]
input_dim

In [ ]:
# Add RUC metric to monitor NN
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim = 31 , kernel_initializer = 'uniform', kernel_regularizer=regularizers.l2(0.005),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,  kernel_initializer = 'uniform',kernel_regularizer=regularizers.l2(0.005),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16,  kernel_initializer = 'uniform',kernel_regularizer=regularizers.l2(0.005),activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_logloss', optimizer='adam', metrics=['accuracy' ,auc])
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
earlystopper = EarlyStopping(monitor='val_loss', patience = 20, verbose=1)
start = time.time() 
model.fit(X_train, y_train, epochs=1000,validation_split=0.2,  batch_size = 512, callbacks=[earlystopper] )
nn_runtime = time.time() - start

In [ ]:
print( ' Neural Net Runtime : {0:.4f}'.format(nn_runtime ))

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve ,auc , log_loss ,  classification_report ,average_precision_score

pred = model.predict(X_test)
pred = pd.DataFrame(data = pred , dtype=np.float32)
y_test =  pd.DataFrame(data = y_test)
y_test.reset_index(inplace = True)
y_test.drop('index',axis=1,inplace=True)
preds = pd.concat([y_test,pred], axis=1)
preds.columns = ['trueLabel','prediction']
# predictionsBasedOnKFoldsXGBoostGradientBoosting = preds.copy()

precision, recall, thresholds = \
    precision_recall_curve(preds['trueLabel'],preds['prediction'])
average_precision = \
    average_precision_score(preds['trueLabel'],preds['prediction'])

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = roc_curve(preds['trueLabel'],preds['prediction'])
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
        Area under the curve = {0:0.2f}'.format(areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
NN_roc_auc_score = roc_auc_score(y_test, model.predict(X_test))

print( 'NeuralNet_ROC_AUC : {0:.4f} '.format(NN_roc_auc_score))